# Montgomery

In [1]:
class Montgomery:
    def __init__(self, m):
        self.m = m
        self.n = m.bit_length()
        self.rrm = (1 << (self.n * 2)) % m
 
    def reduce(self, t):
        a = t
        for i in xrange(self.n):
            if (a & 1) == 1:
                a = a + self.m
            a = a >> 1
        if a >= self.m:
            a = a - self.m
        return a

# CRT

In [2]:
def crt(n, a):
    sum = 0
    product = reduce(lambda a, b: a * b, n)
 
    for n_i, a_i in zip(n, a):
        p = product / n_i
        sum += a_i * mi(p, n_i) * p
    return sum % product

def mi(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a / b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

# Square and Multiply

In [3]:
def square_multiply(x, y):
    exp = bin(y)
    value = x
 
    for i in range(3, len(exp)):
        value = value * value
        if(exp[i : i + 1] == '1'):
            value = value*x
    return value

# Karatsuba

In [4]:
def karatsuba(x, y):
    #função para multiplicar dois números
    if len(str(x)) == 1 or len(str(y)) == 1:
        return x * y
    else:
        n = max(len(str(x)),len(str(y)))
        nby2 = n / 2
        
        a = x / 10 ** (nby2)
        b = x % 10 ** (nby2)
        c = y / 10 ** (nby2)
        d = y % 10 ** (nby2)
        
        ac = karatsuba(a, c)
        bd = karatsuba(b, d)
        #(x1 + x0) (y1 + y0) -x1y1 - x0y0
        ad_bc = karatsuba(a + b, c + d) - ac - bd
        
        #2*nby2
        product = ac * 10 ** (2 * nby2) + (ad_bc * 10 ** nby2) + bd

        return product

In [5]:
import random

#algoritmo de euclides para determinar o maior divisor comum

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

#algoritmo extendido de euclides para encontrar o inverso multiplicativo de dois números

def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi
    
    while e > 0:
        temp1 = temp_phi/e
        temp2 = temp_phi - karatsuba (temp1, e)
        temp_phi = e
        e = temp2
        
        x = x2 - karatsuba(temp1, x1)
        y = d - karatsuba(temp1, y1)
        
        x2 = x1
        x1 = x
        d = y1
        y1 = y
    
    if temp_phi == 1:
        return d + phi

    
#testa se o número é primo

def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in xrange(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('ambos os números precisam ser primos')
    elif p == q:
        raise ValueError('p e q não podem ser iquais')
    #n = pq
    n = karatsuba(p, q)

    phi = karatsuba (p-1, q-1)

    #escolhe um inteiro 'e' tal que 'e' e phi (n) sejam coprimos
    e = random.randrange(1, phi)

    #usa o algoritmo de euclides para verificar se 'e' e phi (n) são coprimos
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #usa o algoritmo extendido de euclides para gerar a chave privada
    d = multiplicative_inverse(e, phi)
    
    #retorna o par de chaves pública e privada
    #chave pública é (e, n) e a chave privada (d, n)
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    #descompacta a chave em seus componentes
    key, n = pk
    
    #converte cada letra no texto simples para números baseados no caractere usando a ^ b mod m
    cipher = [(ord(char) ** key) % n for char in plaintext]
    
    #retorna o array de bytes
    return cipher

def decrypt(pk, ciphertext):
    #descompacta a chave em seus componentes
    key, n = pk
    
    #gera o texto sem formatação baseado no texto cifrado e chave usando a ^ b mod m
    plain = [chr((char ** key) % n) for char in ciphertext]
    
    #retorna o array de bytes como uma string
    return ''.join(plain)
    

if __name__ == '__main__':
    print "RSA"
    p = int(raw_input("enter a prime number: "))
    q = int(raw_input("enter another prime number: "))
    print "generating your public/private keypairs now"
    public, private = generate_keypair(p, q)
    print "your public key is ", public ," and your private key is ", private
    message = raw_input("enter a message to encrypt with your private key: ")
    encrypted_msg = encrypt(private, message)
    print "your encrypted message is: "
    print ''.join(map(lambda x: str(x), encrypted_msg))
    print "decrypting message with public key ", public
    print "your message is:"
    print decrypt(public, encrypted_msg)

RSA
enter a prime number: 17
enter another prime number: 23
generating your public/private keypairs now
your public key is  (327, 391)  and your private key is  (183, 391)
enter a message to encrypt with your private key: lucas da silva costa
your encrypted message is: 
133597410927628012710928027614713318610928074291276346109
decrypting message with public key  (327, 391)
your message is:
lucas da silva costa
